In [1]:

import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd
import numpy as np
import math


In [6]:
df_full_tdrive = pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\tdrive\data\merged_all_security_subset_3km.csv')
df_full_collected = pd.read_csv(r'C:\Users\ss6365\Desktop\Datasets\Collected\filtered_data.csv')

In [3]:
df_full_tdrive

,longitude,latitude,identifier
0,116.39714,39.94757,1
1,116.40070,39.90440,2
2,116.40255,39.91160,2
3,116.40261,39.91164,2
4,116.40044,39.90342,2
...,...,...,...
70757,116.42419,39.93530,6947
70758,116.43078,39.93959,6947
70759,116.43711,39.93974,6947
70760,116.41907,39.94891,6947


In [12]:



your_dataframe = df_full_collected
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    your_dataframe,
    geometry=gpd.points_from_xy(your_dataframe.longitude, your_dataframe.latitude))

# Determine the bounds of your data
bounds = gdf.total_bounds  # returns (minx, miny, maxx, maxy)
map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Your Mapbox access token
mapbox_access_token = 'sk.eyJ1Ijoic2hhZml6dXJyYWhtYW5zZWVhbSIsImEiOiJjbHUwbGt0ZGcwNHI4MmludmNza252aHlzIn0.eQWfr353P4zFLKHiytBOnw'

# Use a predefined Mapbox style
mapbox_style = 'outdoors-v12'  # Example style

# Create the map with the Mapbox style
m = folium.Map(
    location=map_center,
    zoom_start=13,
    tiles='https://api.mapbox.com/styles/v1/mapbox/' + mapbox_style + '/tiles/{z}/{x}/{y}?access_token=' + mapbox_access_token,
    attr='Mapbox'
)

# Create a heatmap
heatmap_data = gdf[['latitude', 'longitude']].values.tolist()
heatmap = HeatMap(heatmap_data, min_opacity=.5, radius=6, blur=20, max_zoom=50)
m.add_child(heatmap)

# Add a rectangle to represent the bounds
folium.Rectangle(
    bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    color=' #000000',
    fill_opacity=1,
    #fill = True,
     weight=2,
).add_to(m)

# Define an outer box with an offset
offset = 0.001  # Adjust as necessary for the desired spacing
outer_bounds = [[bounds[1] - offset, bounds[0] - offset], [bounds[3] + offset, bounds[2] + offset]]

# Create the dotted outer box
folium.PolyLine(
    locations=[outer_bounds[0], [outer_bounds[0][0], outer_bounds[1][1]], outer_bounds[1], [outer_bounds[1][0], outer_bounds[0][1]], outer_bounds[0]],
    color='  red',

    weight=2,
).add_to(m)

# Display the map in the Jupyter Notebook


# Function to adjust longitude size to maintain a square appearance
def adjust_longitude_size(lat, size_lat):
    R = 6378.1  # Earth's radius in kilometers
    lat_rad = math.radians(lat)  # Convert latitude to radians
    size_lon_km = R * math.cos(lat_rad) * math.pi / 180  # Size of 1 degree of longitude in km at the given latitude
    size_lat_km = 111.32  # Approximate size of 1 degree of latitude in km (constant)
    size_lon = size_lat * (size_lat_km / size_lon_km)  # Adjust size in degrees of longitude
    return size_lon

# Assuming 'bounds' contains your GeoDataFrame's total bounds
num_squares_lat = 10 # Number of squares along the latitude
lat_range = np.linspace(bounds[1], bounds[3], num_squares_lat + 1)  # Range of latitudes for the squares

for i in range(num_squares_lat):
    # Average latitude of the current square
    avg_lat = (lat_range[i] + lat_range[i+1]) / 2
    # Size of each square in degrees of latitude
    size_lat = lat_range[i+1] - lat_range[i]
    # Adjusted size of each square in degrees of longitude
    size_lon = adjust_longitude_size(avg_lat, size_lat)
    
    for j in range(num_squares_lat):
        # Longitude range for the squares, adjusted for each latitude
        lon_min = bounds[0] + j * size_lon
        lon_max = bounds[0] + (j + 1) * size_lon
        # Defining the bounds of the current square
        square_bounds = [[lat_range[i], lon_min], [lat_range[i+1], lon_max]]
        
        # Drawing the square on the map
        folium.Rectangle(
            bounds=square_bounds,
            color=' #000000',
            fill=False,
           
            fill_opacity=0.1,
            weight=.6,
        ).add_to(m)

# Display the map
m

# map_html = 'C:\\Users\\ss6365\\Desktop\\VisualCodeImplementation\\main.html'
# m.save(map_html)